In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os,IPython, librosa, mir_eval
from sys import platform
from os import listdir
from os.path import isfile, join,isdir
from IPython.display import Audio
from librosa.display import waveplot,specshow
from librosa.onset import onset_strength, onset_detect
from librosa.feature import melspectrogram, mfcc
from librosa import load

from collections import defaultdict,OrderedDict
import sklearn
from sklearn.preprocessing import StandardScaler,LabelEncoder
import scipy
from pandas import HDFStore,DataFrame



In [2]:
if "Ubuntu" in os.uname().version:
    song, sampling_rate = librosa.load("/home/chib/Documents/fma_small/Electronic/99289.mp3")

#elif (comp == 'Lab'):
#    song, sampling_rate = librosa.load("/Users/chibmac/Documents/fma_small/Electronic/99289.mp3")

else: 
    song, sampling_rate = librosa.load(librosa.util.example_audio_file())


In [3]:
song_length = len(song)
song_length

660984

In [4]:
#number of chunks to split the song into
num_of_sections = 18

#+1 so the last index is included, as python starts at 0
song_splitter = np.arange(0,song_length+1,int(song_length/num_of_sections),dtype=int)
song_splitter

array([     0,  36721,  73442, 110163, 146884, 183605, 220326, 257047,
       293768, 330489, 367210, 403931, 440652, 477373, 514094, 550815,
       587536, 624257, 660978])

In [5]:
#takes each subset of a song (section) as its own song
unit_song = song[song_splitter[0]:song_splitter[1]]
unit_song_length = unit_song.shape[0]
unit_song_length

36721

In [6]:

num_of_mfcc = 12

complete_split = num_of_sections *num_of_mfcc
complete_split

216

In [7]:
window_width = np.int(unit_song_length/num_of_sections)+1

In [8]:
unit_mfcc = librosa.feature.mfcc(y=unit_song, sr=sampling_rate,n_mfcc=num_of_mfcc,hop_length = window_width).T

unit_mfcc.shape

(18, 12)

In [9]:
unit_song.shape

(36721,)

In [10]:
song_to_sect = int(unit_song_length/num_of_sections)
unit_song= unit_song[:num_of_sections*song_to_sect]
split_song = np.reshape(unit_song[:num_of_sections*song_to_sect],(num_of_sections,song_to_sect))
split_song.shape

(18, 2040)

In [11]:
unit_zcr = np.sum(librosa.core.zero_crossings(split_song),axis =1)
unit_zcr

array([112,  54, 110, 204, 112, 128, 139, 104, 172, 156,  69, 112, 123,
        66, 142, 174, 188, 132])

In [12]:
unit_zcr.shape

(18,)

In [13]:
ind_unit_features=np.concatenate((unit_mfcc,np.array([unit_zcr]).T),axis=1)
ind_unit_features.shape

(18, 13)

In [89]:
def complete_extract_features(song,num_of_sections,num_of_mfcc,genre):  
    complete_split = num_of_sections *num_of_mfcc        
    song_splitter = np.arange(0,song_length+1,song_length/num_of_sections,dtype=int)
    genres = np.ones(num_of_sections,dtype=int) * genre
    onset_frames = librosa.onset.onset_detect(y=song, sr=sampling_rate)
    for i in range(len(song_splitter)):
        unit_song = song[song_splitter[i]:song_splitter[i+1]]
        unit_song_length = unit_song.shape[0]
        window_width = np.int(unit_song_length/num_of_sections)+1
        #unit_mfcc = librosa.feature.mfcc(y=unit_song, sr=sampling_rate,n_mfcc=num_of_mfcc,hop_length = window_width).T
        
        split = int(len(unit_song)/num_of_sections)*num_of_sections
        unit_song= unit_song[:split]
    
        split_song = np.reshape(unit_song,(num_of_sections,int(unit_song_length/num_of_sections)))
        unit_zcr = np.sum(librosa.core.zero_crossings(split_song),axis =1)
        #ind_unit_features=np.concatenate((unit_mfcc,np.array([unit_zcr]).T),axis=1)
        ind_unit_features=np.array([unit_zcr]).T
        ind_unit_features = np.reshape(ind_unit_features,(1,np.size(ind_unit_features)))
               
        genre = np.matrix(genre)
        
        #rep_genre =np.array([np.ones(num_of_sections,dtype=int)* 24]).T 
        onset_frames = np.matrix(librosa.onset.onset_detect(y=unit_song, sr=sampling_rate))
        print(onset_frames.shape)
        print(genre.shape)
        print(num_of_sections)
        print(ind_unit_features.shape)
        temp = np.concatenate((ind_unit_features,onset_frames),axis=1)

        #print(full_feat_gen)
        
        cent = np.matrix(librosa.feature.spectral_centroid(y=unit_song, sr=sampling_rate, hop_length= window_width)[0]).mean(axis =1)
        print(cent.shape)
        temp = np.concatenate((temp,cent),axis = 1).T
        oenv = librosa.onset.onset_strength(y=unit_song, sr=sampling_rate, hop_length=window_width)
        tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sampling_rate,
                                      hop_length=window_width)
        print ("ok",temp.shape)
        print(tempogram.shape)
        temp = np.concatenate((temp,tempogram.mean(axis = 1)),axis = 1)
        full_feat_gen = np.concatenate((temp,genre),axis=1)
        #print(tempogram.shape)
        return cent



In [90]:
oenv = librosa.onset.onset_strength(y=unit_song, sr=sampling_rate, hop_length=window_width)
tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sampling_rate,
                              hop_length=window_width)

tempogram.shape

(384, 18)

In [91]:
np.arange(10).reshape(5,2).mean(axis =1 )

array([ 0.5,  2.5,  4.5,  6.5,  8.5])

In [92]:
test_out = complete_extract_features(song,num_of_sections,num_of_mfcc,10)

test_out.shape

(1, 12)
(1, 1)
18
(1, 18)
(1, 1)
ok (31, 1)
(384, 18)


ValueError: all the input arrays must have same number of dimensions

# Song Extraction from fma_small

1. Find the song folder path relative to the current computer
2. Retrieve the different genre classifications
3. Identify each song via its full path to song using index, to guarantee one-to-one mapping 
4. Sort in alphabetical order

In [ ]:
#retrieving path to the fma_small directory and the corresponding meta data
HOME_DIR = IPython.utils.path.get_home_dir()

temp = join(HOME_DIR, 'Documents')
path_to_small_fma = join(temp, 'fma_small')
json_file = join(path_to_small_fma,'fma_small.json')
#locate meta_dta
print(path_to_small_fma)
df = pd.read_json(json_file)
print(json_file)


In [ ]:
#only choose top genre as the label
df = df.loc[:,['top_genre']]

#ensure that the genre name matches file name in fma_small, 
#i.e Oldtime / Historian conflict issue
df['top_genre']=df['top_genre'].apply(lambda y: y.split(os.sep)[0].strip())
#locate each individual song by its full path
df['temp'] = path_to_small_fma
str_index = ["%.2d" % x for x in df.index]
complete_genre_list = df['top_genre'].unique()

#create full path to file and store as a single array
df['full_path_to_song'] = df.temp.map(str)+ "/"+ df['top_genre'].values+ "/"+ str_index+ ".mp3"
del df['temp']
#keep songs according to alphabetical order of songs 
df.sort_values(by = 'top_genre', inplace = True)
df.head()


In [ ]:
complete_genre_list_df = pd.DataFrame(complete_genre_list, columns = ['Genre'])

In [ ]:
#retrieve number of songs per genre
genre_and_count = df['top_genre'].value_counts().sort_index()
all_songs_path = df['full_path_to_song'].values  
print(genre_and_count)

In [ ]:
song_counts = [] 
ordered_genres = [] 
full_song_df = OrderedDict()
genre_to_song_dict = {}


#retrieve number of songs per genre with the order preserved in two lists
#ordered_genres
#song_counts
for i,genre in enumerate(genre_and_count.index):
    ordered_genres.append(genre)
    temp = df['top_genre'].value_counts()[genre]
    song_counts.append(temp)

genre_to_song_zipped = zip(ordered_genres,song_counts)

num_of_genre = np.shape(genre_and_count)[0]
print("The number of genres is:", num_of_genre)

#dictionary with each song count and its corresponding genre
for genre,song_count in genre_to_song_zipped:
    genre_to_song_dict[genre] = song_count

genre_to_song_dict

In [ ]:
num_of_genre = np.shape(genre_and_count)[0]
all_songs_path = df['full_path_to_song'].values

paths_dict = OrderedDict()

#prepend zero so we have a start point for all_songs_path 
#and avoid messing with indices

#use the cumulative sum to find none uniform ranges
song_counts.insert(0,0)
cumulative_sum = np.cumsum(song_counts,dtype=int)

#creates a dictionary of the genres and its corresponding path
for i,genre in enumerate(ordered_genres):
    str1=genre
    str2 = "_paths"
    genre_paths = "".join((str1,str2))
    paths_dict[genre_paths] = all_songs_path[cumulative_sum[i]:cumulative_sum[i+1]]

#paths_dict
#{genre_path_name: genre_paths}
print("{'Electronic_paths:[array_of_all_electronic_paths]}")

In [ ]:
%%time
num_of_songs = 3
sampling_rate = 44100

genre_signals_dict = OrderedDict()
#creates a dictionary of the signals in a genre and their raw file
for genre_path_name,genre_paths in paths_dict.items():
    str1=genre_path_name[:-5]
    str2 = "signals"
    genre_signals = "".join((str1,str2))       
    try:
        first_three = genre_paths[:num_of_songs]
        genre_signals_dict[genre_signals] = [
        load(p,sr=None)[0] for p in first_three]
    except IOError as exc:
        print("Unable to locate folder")
        #raise IOError("%s: %s" % (genre_paths, exc.strerror))
        
#genre_signals_dict
#{genre_signals_name:genre_signals_paths}
print("{'Electronic_signals:[array_of_all_electronic_paths]}")

In [ ]:
#plot the time series for each song according to the genres

#sig_lengths = []
for genre_signal_name,genre_signals in genre_signals_dict.items(): 
    for i, sig_amp in enumerate(genre_signals):
        plt.subplot(1, num_of_songs, i+1)
#        sig_lengths.append(len(sig_amp))
        waveplot(sig_amp)
        plt.ylim(-1, 1)
        plt.title(genre_signal_name)
    plt.figure()
        
    

In [ ]:
song_num = 0
tot_num_of_songs = cumulative_sum[-1]
indiv_song_path= []
for genre_path_name,genre_paths in paths_dict.items(): 
    song_num=song_num+1
    indiv_song_path.append(genre_paths)
  

indiv_song_path_list = np.array(indiv_song_path).reshape(tot_num_of_songs,)

In [ ]:
indiv_song_path_list


In [ ]:
genres = []
for song_num in range(len(indiv_song_path_list)):
    temp = indiv_song_path_list[song_num].split(os.sep)[-2]
    genres.append(temp)
        

In [ ]:
le = LabelEncoder()
encoded_genres= le.fit(genres)
encoded_genres

#Label Encoding Mapping
encoder_df= pd.DataFrame(data= {'Genre':genres,
                   'Encoded_Genre':le.transform(genres)})
encoder_df.head()

In [ ]:
print("I AM HERE")

In [ ]:
%%time
tot_num_of_songs = cumulative_sum[-1]
final_d = np.zeros((tot_num_of_songs,test_out.shape[1]))
print(final_d.shape)

for song_num in range(len(indiv_song_path_list)):
    try:
        indiv_song_path = indiv_song_path_list[song_num]
        song_signal = librosa.load(indiv_song_path,sr=None)[0]
        curr_song_genre = encoder_df['Encoded_Genre'][song_num]
        final_d[song_num]= complete_extract_features(song_signal,num_of_sections,num_of_mfcc,curr_song_genre)

    except IOError as exc:
        print("Unable to locate folder")
    


            

In [ ]:
final_d.shape

In [ ]:
print("I AM HERE")

In [ ]:

complete_df = pd.DataFrame(data=final_d.T,index =range(final_d.shape[1]))
complete_df = complete_df.T

complete_df.head()

In [ ]:
complete_df.shape

In [ ]:
encoder_df.head()

In [ ]:
encoder_df.shape

In [ ]:
final_df = pd.concat([complete_df,encoder_df],axis =1)

In [ ]:
final_df.shape

In [ ]:
final_df = pd.concat([complete_df,encoder_df],axis =1)
final_df.head()


In [ ]:
final_df.shape

In [ ]:
final_df.to_csv("complete_extract_with_onset.csv",sep=',')